# Introduktion
Att spara data för att kunna behandla den senare är ganska bra att kunna göra. För vår del innebär det att vi på någotsätt behöver spara den sensor data som vi provade på att plocka fram i förra labben på ett sätt så vi kan flytta den från vår arduino till vår dator. Våra arduinos har inbyggda SD-korts läsare så vi kommer därför använda oss av SD-kort för att spara vår data. 

## Uppgift 1: Testa SD-kort läsaren

För att kunna använda den inbyggda SD-kort läsaren behövs lite kod som ni gärna får försöka förstå men det är inget måste.

In [ ]:
#Vi börjar med att importera en hel massa bibliotek
import board
import busio
import digitalio
import adafruit_sdcard
import storage

#Startar SD-kortet
spi = busio.SPI(board.SD_SCK, MOSI=board.SD_MOSI, MISO=board.SD_MISO)
cs = digitalio.DigitalInOut(board.SD_CS)
sdcard = adafruit_sdcard.SDCard(spi, cs)
vfs = storage.VfsFat(sdcard)
storage.mount(vfs, "/sd")

with open('/sd/test.txt', 'w') as file:
    file.write('Du har skrivit data till ett SD-kort')

Prova att köra koden ovan, koppla sedan in SD-kortet i din dator och se till så att filen faktiskt finns där. Prova också att skriva något annat till filen eller prova att döpa filen till nått annat.

## Mäta och spara data

I förra labben lärde ni er att mäta olika typer av värden på olika fenomen. Ni har nu också lärt er hur data kan sparas ned på en fil. Det är nu dags att kombinera dessa två. Det gör vi genom att koppla in accelerometern på samma sätt som i föregående labb. 

### ADXL345

Börja med att koppla ihop sensorn och arduinon enligt tabellen, precis som i förra labben.

| Arduino pin | Sensor pin |
|-------------|------------|
|      GND    |     GND    |
|      VCC    |     VCC    |
|      SDA    |     SDA    |
|      SCL    |     SCL    |

Sen behöver vi lite kod.


In [ ]:
import time
import board
import busio
import digitalio
import adafruit_sdcard
import storage
import adafruit_adxl34x

#Starta sensorn
i2c = busio.I2C(board.SCL, board.SDA)
accelerometer = adafruit_adxl34x.ADXL345(i2c)

#Starta SD-kortet
spi = busio.SPI(board.SD_SCK, board.SD_MOSI, board.SD_MISO)
cs = digitalio.DigitalInOut(board.SD_CS)
sdcard = adafruit_sdcard.SDCard(spi, cs)
vfs = storage.VfsFat(sdcard)
storage.mount(vfs, "/sd")

# Skapa och öppna en fil för att spara datan
with open("/sd/log.txt", "a") as logfile:
    while True:

        # Läs datan
        acceleration = accelerometer.acceleration
        x, y, z = acceleration
        print(f"x: {x:.2f}, y: {y:.2f}, z: {z:.2f}")

        # Skriv datan till filen
        logfile.write(f"{time.time()},{x:.2f},{y:.2f},{z:.2f}\n")
        logfile.flush()

        # Fördröjning till nästa mätning
        time.sleep(1)


### DHT11
Börja med att koppla in sensorn enligt tabellen, precis som förra gången.

| Arduino pin | Sensor pin |
|-------------|------------|
|      GND    |     GND    |
|      VCC    |     VCC    |
| Vilken som helst t.ex 2   |   Signal   |

Sen behövs lite kod

In [ ]:
import time
import board
import busio
import digitalio
import adafruit_sdcard
import storage
import adafruit_dht

# Starta sensorn
dht_device = adafruit_dht.DHT11(board.D2)

# Starta SD-kortet
spi = busio.SPI(board.SD_SCK, board.SD_MOSI, board.SD_MISO)
cs = digitalio.DigitalInOut(board.SD_CS)
sdcard = adafruit_sdcard.SDCard(spi, cs)
vfs = storage.VfsFat(sdcard)
storage.mount(vfs, "/sd")

#Skapar och/eller öppnar en fil log.txt på SD-kortet
with open("/sd/log.txt", "a") as logfile:
    while True:
        try:
            #Läs datan
            temperature = dht_device.temperature
            humidity = dht_device.humidity
            print(f"Temperature: {temperature:.1f} C, Humidity: {humidity:.1f} %")

            #Skriv till filen
            logfile.write(f"{time.time()},{temperature:.1f},{humidity:.1f}\n")
            logfile.flush()  # Ensure data is written to the file

        except RuntimeError as error: #DHT11 sensorn är ganska opålitlig så för att koden inte ska sluta köra bara för att den inte råkar funka en cykel behövs denna koden
            print(error.args[0])

        #Fördröjning till nästa mätning
        time.sleep(2)


Nu när vi vet hur vi skriver data till SD-kort behöver vi lära oss hur vi lagrar datan så den är lätt att använda senare. Det finns en gyllene standarda som används överallt som kallas CSV filer. CSV står för comma separated values. Det är alltså en filtyp som innehåller värden separerade med kommatecken. Titta på exempel filen och se hur den ser ut. Längst upp i filen finns en header, där hittar vi namnen på kolumnerna som följer. I en CSV fil är headern inte ett måste men är ganska trevligt att ha för att hålla koll på sin data. Anledningen till att man använder CSV filer är för att dom är mycket enkla att läsa av senare som ni kommer se i nästa del av labben. 

I nästa del av labben ska ni använda python för att skapa lite grafer med data ni samlat in. Så nu ska vi fixa lite data. Först ska ni få lite kod för att skriva data till CSV filer. Nedan följer ett exepel för DHT11 sensorn men ni kan modifiera koden för att skriva vilken data som helst.

In [ ]:
import csv #python bibliotek för att skapa, skriva och hantera csv filer

with open("/sd/log.csv", "a", newline='') as logfile: #Skapa och/eller öppna filen log.csv på SD-kortet
    fieldnames = ["timestamp", "temperature", "humidity"] #Här skriver vi vilka kolumn namn vi vill ha
    writer = csv.DictWriter(logfile, fieldnames=fieldnames) #Skapar ett skrivar objekt så vi kan skriva till filen
    
    
    if logfile.tell() == 0: #Kollar om det finns en header i filen redan
        writer.writeheader() #Skriver headern till filen om det inte finns en

    while True:
        try:
            # Läser data från sensorn
            temperature = dht_device.temperature
            humidity = dht_device.humidity
            print(f"Temperature: {temperature:.1f} C, Humidity: {humidity:.1f} %")#Skriver ut datan i serial-monitorn

            #Skriver datan till csv filen
            writer.writerow({"timestamp": time.time(), "temperature": temperature, "humidity": humidity})
            logfile.flush()

        except RuntimeError as error:
            print(error.args[0])

        # Delay before next reading
        time.sleep(2)

Nu när vi kan skriva data till csv filer kan vi skapa lite rolig data att plotta. Så välj en sensor och lite aktiviteter att göra med sensor som kan ge intressanta utslag på en graf. Genomför era roliga aktiviteter överför datan från SD-kortet till er dator och ladda upp det på er jupyternotebook och hoppa till nästa notebook.